In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.optimize import curve_fit
from scipy.signal import find_peaks

In [4]:
os.chdir('C:\\Users\\luukj\\Documents\\GitHub\\Project-2.1\\metingen\\metingen suiker\\0.5')

# constants
lambda_laser = 532e-9 # wavelength of the laser in meters

d_sugar= 1e-2 # thickness in meters
d_cuvett = 1.25e-2 - d_sugar # thickness in meters

n_cuvett = 1.59 # refractive index of the cuvett
n_air = 1.0 # refractive index of air

window_size = 5
prom = 0.5

In [5]:
def read(name):
    df = pd.read_csv(
    str(name) +'.csv',
    skiprows=2,
    delimiter=',',
    names=['X', 'CH1', 'dummy1', 'dummy2'],
    engine='python'
    )
    return df

def N_fit_suiker(theta_in,n_sugar):
    """Function to fit the refractive index data."""
    theta_in_rad = np.deg2rad(theta_in)
    return 2/lambda_laser*(2*d_cuvett*(np.sqrt(n_cuvett**2 -n_air**2*np.sin(theta_in_rad)**2)-n_air*np.cos(theta_in_rad)) + d_sugar*(np.sqrt(n_sugar**2 -n_air**2*np.sin(theta_in_rad)**2)-n_air*np.cos(theta_in_rad))-(2*n_cuvett*d_cuvett + n_sugar*d_sugar - 2*d_cuvett*n_air-d_sugar*n_air))

In [ ]:
df02 = read('0.5ml_0g2g')
df24 = read('0.5ml_2g4g')
df46 = read('0.5ml_4g6g')
df68 = read('0.5ml_6g8g')
df810 = read('0.5ml_8g10g')
# Rolling averages toevoegen
df02['rolling'] = df02['CH1'].rolling(window=window_size, center=True).mean()
df24['rolling'] = df24['CH1'].rolling(window=window_size, center=True).mean()
df46['rolling'] = df46['CH1'].rolling(window=window_size, center=True).mean()
df68['rolling'] = df68['CH1'].rolling(window=window_size, center=True).mean()
df810['rolling'] = df810['CH1'].rolling(window=window_size, center=True).mean()

peaks_02, props_02 = find_peaks(df02['rolling'], prominence=prom)
peaks_24, props_24 = find_peaks(df24['rolling'], prominence=prom)
peaks_46, props_46 = find_peaks(df46['rolling'], prominence=prom)
peaks_68, props_68 = find_peaks(df68['rolling'], prominence=prom)
peaks_810, props_810 = find_peaks(df810['rolling'], prominence=prom)

fringes_0 = 0
fringes_02 = fringes_0 + len(peaks_02)
fringes_24 = fringes_02 + len(peaks_24)
fringes_46 = fringes_24 + len(peaks_46)
fringes_68 = fringes_46 + len(peaks_68)
fringes_810 = fringes_68 + len(peaks_810)
fringes = np.array([fringes_0,fringes_02,fringes_24,fringes_46,fringes_68,fringes_810])

theta_graden = np.array([0,2,4,6,8,10])

ValueError: operands could not be broadcast together with shapes (12,) (20,) 

In [ ]:
fringes_0 = 0
fringes_02 = fringes_0 + (peaks_02)
fringes_24 = fringes_02 + peaks_24
fringes_46 = fringes_24 + peaks_46
fringes_68 = fringes_46 + peaks_68
fringes_810 = fringes_68 + peaks_810
fringes = np.array([fringes_0,fringes_02,fringes_24,fringes_46,fringes_68,fringes_810])

In [ ]:
# Plotten
plt.figure(0)
plt.plot(df02['X'], df02['CH1'], alpha=0.5, label='Origineel')
plt.plot(df02['X'], df02['rolling'], color='red', label=f'Rolling avg ({window_size})')
plt.title('0 → 2')
plt.legend()

plt.figure(1)
plt.plot(df24['X'], df24['CH1'], alpha=0.5, label='Origineel')
plt.plot(df24['X'], df24['rolling'], color='red', label=f'Rolling avg ({window_size})')
plt.title('2 → 4')
plt.legend()

plt.figure(2)
plt.plot(df46['X'], df46['CH1'], alpha=0.5, label='Origineel')
plt.plot(df46['X'], df46['rolling'], color='red', label=f'Rolling avg ({window_size})')
plt.title('4 → 6')
plt.legend()

plt.figure(3)
plt.plot(df68['X'], df68['CH1'], alpha=0.5, label='Origineel')
plt.plot(df68['X'], df68['rolling'], color='red', label=f'Rolling avg ({window_size})')
plt.title('6 → 8')
plt.legend()

plt.figure(4)
plt.plot(df810['X'], df810['CH1'], alpha=0.5, label='Origineel')
plt.plot(df810['X'], df810['rolling'], color='red', label=f'Rolling avg ({window_size})')
plt.title('8 → 10')
plt.legend()

In [ ]:
# fit the data
popt, pcov = curve_fit(N_fit_suiker, theta_graden, fringes , p0=[1.3])
n_fit = popt
theta_fit = np.linspace(0,12,100)
n_line = N_fit_suiker(theta_fit, *n_fit)

# plotting
plt.figure(0)
plt.scatter(theta_graden, fringes, label='Measured Data', color='red')
plt.plot(theta_fit, n_line, label=f'Fitted Curve (n={n_fit[0]:.2f})', color='blue')
plt.plot(theta_fit,N_fit_suiker(theta_fit,1.35),label='n = 1.35',color='orange')
plt.xlabel('Theta (degrees)')
plt.ylabel('N')
plt.title('Refractive Index Fit')
plt.xlim(0,12)
plt.ylim(0,200)          
plt.legend()
plt.grid()